- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

## TOPIC Name : Sentiment Analysis using AutoML: Deployment

#### Author: Arun Sharma K

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

In [1]:
!pip install gradio #library used to create simple webapp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio
from google.colab import drive
import os
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
drive.mount('/content/drive')
os.chdir( "/content/drive/My Drive/GL_Deeplearning_notebooks/NLP_w1")

Mounted at /content/drive


In [4]:
def cleanup_reviews(raw_review):
    # 1. Remove HTML tags
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()

    # 2. Remove non-alphabetic charecters using regex
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()

    # 4. Create set of stopwords
    stops = set(stopwords.words("english"))

    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]

    # 6. Join the words back into one string separated by space,
    # and return the result.
    return( " ".join( meaningful_words ))

In [5]:
# this function removed the words present in 'stops' from the 'raw_text'
def remove_commonwords(raw_text, stops=""):
    words = raw_text.split()
    meaningful_words = [w for w in words if not w in stops]
    return( " ".join( meaningful_words ))

In [6]:
def get_sentiment(text):
  if text=="":
    sentiment="No input"
  else:
      com_freq_words=['one', 'even', 'character', 'seem', 'sceen','get', 'film','see','like', 'seen', 'look', 'thing','movie', 'time', 'show', 'good','first','made', 'story','much','would','also','well','people', 'scene', 'movies', 'really','make','think', 'ways', 'characters', 'could']
      vocab=pickle.load(open("vectorizer.pkl", "rb"))
      clean_txt=cleanup_reviews(text)
      clean_txt=remove_commonwords(clean_txt,com_freq_words)
      vectorizer=TfidfVectorizer(vocabulary=vocab)
      features=vectorizer.fit_transform([clean_txt])
      lr_clf=pickle.load(open("tuned_lr.pkl", "rb"))
      pred= lr_clf.predict(features)
      if pred==1:
        sentiment= 'Positive'
      else:
        sentiment = 'Negative'
  return sentiment


In [7]:
gradio.Interface(get_sentiment, inputs="text",  outputs="text", title="Predict sentiment").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aab37ec06380921dd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import tensorflow as tf
import numpy as np

# Create a simple model for illustration
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

# Assume the model is already trained (skipping that part here)

# Example input data: 2 samples, each with 4 features
input_data = np.array([[1.0, 2.0, 3.0, 4.0]])

# Predicting the output
predictions = model.predict(input_data)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step


In [9]:
predictions

array([[0.58445543, 0.13837467, 0.26043186, 0.00515143, 0.01158666]],
      dtype=float32)

In [10]:
import numpy as np

class_labels=["A", "B","C", "D", "E"]

In [11]:
def get_label_from_prob(model_predict, class_labels):
  pos=np.argmax(model_predict)
  print(pos)
  label = class_labels[pos]
  return label

In [12]:
input_data = np.array([[1.0, 2.0, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0]])
predictions = model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step


In [13]:
predictions

array([[5.8445543e-01, 1.3837466e-01, 2.6043186e-01, 5.1514297e-03,
        1.1586660e-02],
       [6.5873766e-01, 1.6960683e-01, 1.7155780e-01, 1.8008677e-05,
        7.9678372e-05]], dtype=float32)

In [14]:
for i, prediction in enumerate(predictions):
  print (prediction)
  label = get_label_from_prob(prediction,class_labels)
  print("Predicted Class Labels:", label)

[0.58445543 0.13837466 0.26043186 0.00515143 0.01158666]
0
Predicted Class Labels: A
[6.5873766e-01 1.6960683e-01 1.7155780e-01 1.8008677e-05 7.9678372e-05]
0
Predicted Class Labels: A
